In [1]:
import pandas as pd
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [67]:
with open("swear_words.txt") as f:
    profanity = f.read().splitlines() 

In [81]:
df = pd.read_csv('reviews.csv')
#df = pd.read_csv('pre_test.csv')

In [84]:
def swear_words(x):
    count=0
    words = x.translate(str.maketrans('', '', string.punctuation)).split(" ")
    for word in words:
        if word.lower() in profanity:
            count+=1
    return count

def num_words(x):
    return len(x.split(" "))

def count_ques(x):
    count = 0
    for char in x:
        if char == "?":
            count +=1
    return count

def count_exclam(x):
    count = 0
    for char in x:
        if char == "!":
            count +=1
    return count

def find_caps(x):
    count = 0
    words = x.translate(str.maketrans('', '', string.punctuation)).split(" ")
    for word in words:
        if len(word) >2 and word.isupper():
            count += 1
    return count

In [89]:
lemmatizer = WordNetLemmatizer()

In [90]:
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None
 
# For each noun, adjective, and adverb in a review, evaluate the positive and negative scores
# of its first synset and average the differences across the review
def swn_polarity(text):
    sentiment = 0.0
    tokens_count = 0
    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
    # default to negative
    if not tokens_count:
        return 0
    return sentiment/tokens_count

In [91]:
df['avg_sent'] = df['review'].apply(swn_polarity)

In [93]:
df['num_words'] = df['review'].apply(num_words)

In [94]:
def said_i(x):
    count = 0
    words = x.translate(str.maketrans('', '', string.punctuation)).split(" ")
    for word in words:
        if word.lower() == "i":
            count += 1
    return count

In [95]:
df['said_i'] = df['review'].apply(said_i)/df['num_words']
df['avg_caps'] = df['review'].apply(find_caps)/df['num_words']
df['avg_ques'] = df['review'].apply(count_ques)/df['num_words']
df['avg_exclam']= df['review'].apply(count_exclam)/df['num_words']
df['avg_swear'] = df['review'].apply(swear_words)/df['num_words']

In [97]:
df2 = df[['pos','avg_sent','avg_ques','avg_caps','avg_exclam','num_words','said_i','avg_swear']]

In [98]:
df2.head()

,pos,avg_sent,avg_ques,avg_caps,avg_exclam,num_words,said_i,avg_swear
0,0,0.007143,0.000000,0.000000,0.0,94,0.010638,0.0
1,0,-0.103365,0.006993,0.000000,0.0,143,0.048951,0.0
2,0,-0.068966,0.000000,0.000000,0.0,177,0.000000,0.0
3,0,-0.091549,0.000000,0.005525,0.0,181,0.000000,0.0
4,0,0.019366,0.000000,0.000000,0.0,212,0.028302,0.0


In [99]:
df2.to_csv("train2.csv",encoding='utf-8',index=False)